<a href="https://colab.research.google.com/github/dakilaledesma/VIGOR2/blob/main/notebooks/%5BNI%5D_Kinematics_to_Music_Protomodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
! curl -L https://herbarium.ucht.cloud/s/AoZFbaKWofEfHiF/download --output motions.zip
! curl -L https://herbarium.ucht.cloud/s/Cn55KNbEBqtrDCg/download --output mp3.zip
! curl -L https://herbarium.ucht.cloud/s/MayD6rAGWoKmGw8/download --output keypoints3d.zip
! unzip -q motions.zip
! unzip -q keypoints3d.zip
! unzip -q mp3.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  305M  100  305M    0     0  30.0M      0  0:00:10  0:00:10 --:--:-- 46.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.6M  100 90.6M    0     0  25.1M      0  0:00:03  0:00:03 --:--:-- 25.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  834M  100  834M    0     0  45.6M      0  0:00:18  0:00:18 --:--:-- 48.3M


In [1]:
audio_dict = {}
motion_dict = {}

In [4]:
# Thanks to https://stackoverflow.com/questions/69387104/how-to-convert-wav-audio-file-from-mel-spectrogram
import librosa
import os
from glob import glob

for aud in glob("mp3/*.mp3"):
  my_audio_as_np_array, my_sample_rate = librosa.load(aud)

  spec = librosa.feature.melspectrogram(y=my_audio_as_np_array,
                                          sr=my_sample_rate, 
                                              n_fft=2048, 
                                              hop_length=512, 
                                              win_length=None, 
                                              window='hann', 
                                              center=True, 
                                              pad_mode='reflect', 
                                              power=2.0,
                                      n_mels=128)
  spec = spec.T[:1272].T # Minimum is (128, 1273)
  bn = os.path.basename(aud)
  audio_dict[aud] = spec

In [5]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

for fn in glob("keypoints3d/*.pkl"):
  with open(fn, "rb") as motion:
    m = pickle.load(motion)
    bn = os.path.basename(fn)
    motion_dict[bn] = m["keypoints3d"][:425] # Minimum is (426, 17, 3), axis=0 is all different

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import layers

input_img = keras.Input(shape=(426, 17, 3))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
res = librosa.feature.inverse.mel_to_audio(spec, 
                                           sr=my_sample_rate, 
                                           n_fft=2048, 
                                           hop_length=512, 
                                           win_length=None, 
                                           window='hann', 
                                           center=True, 
                                           pad_mode='reflect', 
                                           power=2.0, 
                                           n_iter=32)

import soundfile as sf
sf.write("test1.wav", res, sashi_sr)